In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/us-consumer-finance-complaints/database.sqlite
/kaggle/input/us-consumer-finance-complaints/consumer_complaints.csv


### Dataset being used: US Consumer Finance Complaints

In [3]:
df = pd.read_csv("../input/us-consumer-finance-complaints/consumer_complaints.csv")
df.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067


In [4]:
df.shape

(555957, 18)

In [5]:
subset_df = df[['product', 'sub_product', 'issue', 'sub_issue', 'consumer_complaint_narrative', 'company', 'company_response_to_consumer']]
subset_df.head()

,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company,company_response_to_consumer
0,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,U.S. Bancorp,Closed with explanation
1,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,Wells Fargo & Company,Closed with explanation
2,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,Wells Fargo & Company,Closed with explanation
3,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,"Navient Solutions, Inc.",Closed with explanation
4,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,Resurgent Capital Services L.P.,Closed with explanation


In [11]:
subset_df['company'].value_counts()

Bank of America                           55998
Wells Fargo & Company                     42024
JPMorgan Chase & Co.                      33881
Equifax                                   31828
Experian                                  30905
                                          ...  
United Services Veterans Mortgage, LLC        1
Sher & Shabsin PC                             1
HAC Holding Inc.                              1
Mezzetti Financial Services, Inc.             1
Ashner Group Inc                              1
Name: company, Length: 3605, dtype: int64

In [12]:
subset_df['company'].nunique()

3605

In [6]:
df['company_response_to_consumer'].value_counts()

Closed with explanation            404293
Closed with non-monetary relief     70237
Closed with monetary relief         38262
Closed without relief               17909
Closed                              13399
Closed with relief                   5305
In progress                          3763
Untimely response                    2789
Name: company_response_to_consumer, dtype: int64

## EDA

## Questions to answer from Tableau:

1. Which product and sub-product had the highest complaints
2. Which company recieved the highest complaint and in what product
3. Which company and what products were closed unexpectdely or are in progress
4. Which date/year were maximum complaints lodged company wise

In [21]:
# Here we need to focus on topic modelling of consumer complaints and analyzing how good the topics were.
# subseting the data with consumer complaints narrative present

narrative_df = subset_df[['consumer_complaint_narrative']]
narrative_df = narrative_df.dropna()
narrative_df.head()

,consumer_complaint_narrative
190126,XXXX has claimed I owe them {$27.00} for XXXX ...
190135,Due to inconsistencies in the amount owed that...
190155,In XX/XX/XXXX my wages that I earned at my job...
190207,I have an open and current mortgage with Chase...
190208,XXXX was submitted XX/XX/XXXX. At the time I s...


## Text Cleaning

In [19]:
from nltk.tokenize import word_tokenize
# from nltk.tokenize import TweetTokenizer -- specifically made for tweets to remove @, #, emoticons
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer


def clean_text(text):
    
    # tokenize
    tokens = word_tokenize(str(text))
    
    # lower case
    tokens = [w.lower() for w in tokens]
    
    # remove punctuation
    punctuations = string.punctuation
    tokens = [w for w in tokens if w not in punctuations]
    
    # removing tokens that are not alphabetic
    tokens = [w for w in tokens if not w.isdigit()]
    
    # remove stop words
    ADDITIONAL_STOPWORDS = ['hello']
    stop_words = set(stopwords.words('english') + ADDITIONAL_STOPWORDS)
    tokens = [w for w in tokens if w not in stop_words]
    
    # stem
    porter = PorterStemmer()
    tokens = [porter.stem(w) for w in tokens]
    
    # lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    
    
    return tokens
    

# Test String
test_string = "Hello, my name is Prakriti, my email has @, I like Sketching and have 2 cats"
cleaned_string = clean_text(test_string)
cleaned_string

['name', 'prakriti', 'email', 'like', 'sketch', 'cat']

In [17]:
narrative_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66806 entries, 190126 to 553096
Data columns (total 1 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   consumer_complaint_narrative  66806 non-null  object
dtypes: object(1)
memory usage: 1.0+ MB


In [14]:
narrative_df.describe()

,consumer_complaint_narrative
count,66806
unique,65646
top,This company continues to report on my credit ...
freq,37


In [16]:
# to convert the column of objects to string - not required here because I included this while tokenizing
narrative_df.consumer_complaint_narrative.apply(str)

190126    XXXX has claimed I owe them {$27.00} for XXXX ...
190135    Due to inconsistencies in the amount owed that...
190155    In XX/XX/XXXX my wages that I earned at my job...
190207    I have an open and current mortgage with Chase...
190208    XXXX was submitted XX/XX/XXXX. At the time I s...
                                ...                        
553084    XXXX XXXX is reporting incorrectly, payments h...
553085    Reflecting incorrect payment status. Have been...
553086    I have been paying {$180.00} a month through d...
553090    I recently became aware that Amerisave Mortgag...
553096    Bank of America has demonstrated an on-going l...
Name: consumer_complaint_narrative, Length: 66806, dtype: object

In [23]:
## In the interest of time, taking only 10k complaints for now
narrative_df = narrative_df[0:10000]
narrative_df.shape

(10000, 1)

In [25]:
narrative_df['complaints_tokenized'] = df['consumer_complaint_narrative'].apply(lambda x: clean_text(x))
narrative_df.head()

,consumer_complaint_narrative,complaints_tokenized
190126,XXXX has claimed I owe them {$27.00} for XXXX ...,"[xxxx, claim, owe, 27.00, xxxx, year, despit, ..."
190135,Due to inconsistencies in the amount owed that...,"[due, inconsist, amount, owe, told, bank, amou..."
190155,In XX/XX/XXXX my wages that I earned at my job...,"[xx/xx/xxxx, wage, earn, job, decreas, almost,..."
190207,I have an open and current mortgage with Chase...,"[open, current, mortgag, chase, bank, xxxx, ch..."
190208,XXXX was submitted XX/XX/XXXX. At the time I s...,"[xxxx, submit, xx/xx/xxxx, time, submit, compl..."


## Bag of Words Model

In [26]:
import gensim

processed_docs = narrative_df.complaints_tokenized
dictionary = gensim.corpora.Dictionary(processed_docs)

# removing too frequent and too rare words
# no_below: words appearing less than 10 times in the corpus
# no_above: Words appearing more than 50% of the times in the corpus
# keep_n: keep only the first n most frequent tokens
dictionary.filter_extremes(no_below = 10, no_above = 0.5, keep_n = 100000)

# converting Dict to BOW
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [30]:
# printing bag of words of a specific complaint

bow_doc = bow_corpus[1190]

for i in range(len(bow_doc)):
    print("Word {} (\"{}\") appears {} times".format(bow_doc[i][0], dictionary[bow_doc[i][0]], bow_doc[i][1]))

Word 3 ("bill") appears 12 times
Word 15 ("paid") appears 1 times
Word 16 ("payment") appears 10 times
Word 27 ("amount") appears 7 times
Word 28 ("balanc") appears 4 times
Word 31 ("contact") appears 4 times
Word 33 ("credit") appears 6 times
Word 37 ("due") appears 4 times
Word 42 ("given") appears 1 times
Word 50 ("made") appears 1 times
Word 55 ("offer") appears 1 times
Word 56 ("one") appears 1 times
Word 60 ("pay") appears 7 times
Word 61 ("per") appears 2 times
Word 64 ("receiv") appears 4 times
Word 65 ("remain") appears 3 times
Word 69 ("request") appears 2 times
Word 73 ("show") appears 1 times
Word 74 ("still") appears 1 times
Word 77 ("told") appears 3 times
Word 78 ("total") appears 1 times
Word 83 ("write") appears 1 times
Word 84 ("yet") appears 1 times
Word 85 ("100.00") appears 1 times
Word 87 ("account") appears 2 times
Word 92 ("also") appears 2 times
Word 95 ("appli") appears 2 times
Word 98 ("back") appears 1 times
Word 99 ("base") appears 2 times
Word 103 ("call")

## LDA using BOW